In [ ]:
from langsmith import Client
import os

os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_API_KEY"] = "{YOUR_LANGCHAIN_APIKEY}"
os.environ["OPENAI_API_KEY"] = "{YOUR_OPENAI_KEY}"

import langsmith

from langchain import prompts
from langchain.schema import output_parser
from langchain_openai import ChatOpenAI
from langchain.smith import RunEvalConfig


# Define your runnable or chain below.
prompt = prompts.ChatPromptTemplate.from_messages(
    [("system", "You are a helpful AI assistant."), ("human", "{question}")]
)
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
chain = prompt | llm | output_parser.StrOutputParser()

# Define the evaluators to apply
eval_config = RunEvalConfig(
    evaluators=[
        "cot_qa",
        RunEvalConfig.LabeledCriteria("conciseness"),
        RunEvalConfig.LabeledCriteria("relevance"),
        RunEvalConfig.LabeledCriteria("coherence"),
        RunEvalConfig.LabeledCriteria("harmfulness"),
        RunEvalConfig.LabeledCriteria("maliciousness"),
        RunEvalConfig.LabeledCriteria("helpfulness"),
        RunEvalConfig.LabeledCriteria("controversiality"),
        RunEvalConfig.LabeledCriteria("misogyny"),
        RunEvalConfig.LabeledCriteria("criminality"),
        RunEvalConfig.LabeledCriteria("insensitivity"),
    ],
    custom_evaluators=[],
    eval_llm=llm,
)

client = Client()
chain_results = client.run_on_dataset(
    dataset_name="Rap Battle Dataset 1",
    llm_or_chain_factory=chain,
    evaluation=eval_config,
    project_name="test-advanced-fly-10",
    concurrency_level=5,
    verbose=True,
)
chain_results